### MODEL 1

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import joblib
from tensorflow.keras.models import load_model

try:
    df = pd.read_csv('training_set.csv')
    print("Training dataset loaded successfully.")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'training_set.csv' not found. Please place it in the correct directory.")
    exit()

Training dataset loaded successfully.
Dataset shape: (1981520, 79)


In [9]:
print("\nMissing values per column:")
print(df.isnull().sum())

for col in df.select_dtypes(include=np.number).columns:
    df[col].fillna(df[col].mean(), inplace=True)

X = df.drop(' Label', axis=1)
y = df[' Label']


Missing values per column:
 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64


/tmp/ipython-input-46994758.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [10]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)

scaler1 = MinMaxScaler()
X_scaled = scaler1.fit_transform(X)

print("Features have been normalized using Min-Max scaling (range 0 to 1).")
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

joblib.dump(scaler1, 'scaler1.gz')

/tmp/ipython-input-3750361215.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)


Features have been normalized using Min-Max scaling (range 0 to 1).


['scaler1.gz']

In [11]:
y = y.astype(int)

weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = dict(enumerate(weights))

print(f"\nCalculated class weights: {class_weights}")
print("This will penalize errors on the minority class more heavily.")
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

model = tf.keras.Sequential([

    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

print("\n--- Starting Model Training with Early Stopping and Checkpointing ---")
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

print("--- Model Training Finished ---")


Calculated class weights: {0: np.float64(0.6226620947630923), 1: np.float64(2.5381194409148664)}
This will penalize errors on the minority class more heavily.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         5,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,169 (28.00 KB)

...


--- Starting Model Training with Early Stopping and Checkpointing ---
Epoch 1/5
49529/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9328 - loss: 0.1506 - precision: 0.7668 - recall: 0.9544
Epoch 1: val_loss improved from inf to 0.07889, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 143s 3ms/step - accuracy: 0.9328 - loss: 0.1506 - precision: 0.7668 - recall: 0.9544 - val_accuracy: 0.9699 - val_loss: 0.0789 - val_precision: 0.8799 - val_recall: 0.9810
Epoch 2/5
49532/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9640 - loss: 0.0886 - precision: 0.8592 - recall: 0.9776
Epoch 2: val_loss improved from 0.07889 to 0.07072, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - accuracy: 0.9640 - loss: 0.0886 - precision: 0.8592 - recall: 0.9776 - val_accuracy: 0.9692 - val_loss: 0.0707 - val_precision: 0.8747 - val_recall: 0.9850
Epoch 3/5
49519/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9678 - loss: 0.0791 - precision: 0.8716 - recall: 0.9811
Epoch 3: val_loss improved from 0.07072 to 0.06126, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 138s 3ms/step - accuracy: 0.9678 - loss: 0.0791 - precision: 0.8716 - recall: 0.9811 - val_accuracy: 0.9764 - val_loss: 0.0613 - val_precision: 0.9032 - val_recall: 0.9860
Epoch 4/5
49530/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9692 - loss: 0.0763 - precision: 0.8759 - recall: 0.9827
Epoch 4: val_loss did not improve from 0.06126
49538/49538 ━━━━━━━━━━━━━━━━━━━━ 136s 3ms/step - accuracy: 0.9692 - loss: 0.0763 - precision: 0.8759 - recall: 0.9827 - val_accuracy: 0.9728 - val_loss: 0.0682 - val_precision: 0.8847 - val_recall: 0.9910
Epoch 5/5
49524/49538 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9707 - loss: 0.0729 - precision: 0.8807 - recall: 0.9838
Epoch 5: val_loss improved from 0.06126 to 0.06125, saving model to best_model.h5


49538/49538 ━━━━━━━━━━━━━━━━━━━━ 138s 3ms/step - accuracy: 0.9707 - loss: 0.0729 - precision: 0.8807 - recall: 0.9838 - val_accuracy: 0.9771 - val_loss: 0.0613 - val_precision: 0.9029 - val_recall: 0.9903
Restoring model weights from the end of the best epoch: 5.
--- Model Training Finished ---


In [12]:
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)

print("\n--- Model Evaluation on Validation Set ---")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

12385/12385 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step

--- Model Evaluation on Validation Set ---
Confusion Matrix:
[[309919   8315]
 [   755  77315]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99    318234
           1       0.90      0.99      0.94     78070

    accuracy                           0.98    396304
   macro avg       0.95      0.98      0.97    396304
weighted avg       0.98      0.98      0.98    396304



In [13]:
try:
    test_df = pd.read_csv('test_set.csv')
    print("Hold-out test set loaded successfully.")
except FileNotFoundError:
    print("Error: 'testing_set.csv' not found. Please check the file name.")
    exit()

X_test = test_df.drop(' Label', axis=1)
y_test = test_df[' Label']
print("--- Applying preprocessing to the test set ---")

train_cols = X.columns
X_test = X_test.reindex(columns=train_cols, fill_value=0)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.fillna(X.median(), inplace=True)
X_test_scaled = scaler1.transform(X_test)

print("Test set preprocessed identically.")
print("\n--- Generating Final Performance Report ---")

y_test_pred_proba = model.predict(X_test_scaled)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

print("\nFINAL Classification Report on Unseen Test Data:")
print(classification_report(y_test, y_test_pred))

print("\nFINAL Confusion Matrix on Unseen Test Data:")
print(confusion_matrix(y_test, y_test_pred))

Hold-out test set loaded successfully.
--- Applying preprocessing to the test set ---
Test set preprocessed identically.

--- Generating Final Performance Report ---
26539/26539 ━━━━━━━━━━━━━━━━━━━━ 38s 1ms/step

FINAL Classification Report on Unseen Test Data:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99    681929
           1       0.91      0.99      0.95    167294

    accuracy                           0.98    849223
   macro avg       0.95      0.98      0.97    849223
weighted avg       0.98      0.98      0.98    849223


FINAL Confusion Matrix on Unseen Test Data:
[[664559  17370]
 [  1662 165632]]


### MODEL 2

In [14]:
try:
    df = pd.read_csv('secondary_classification_dataset.csv')
    print("Dataset loaded successfully.")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'your_multiclass_dataset.csv' not found. Please check the file name.")
    exit()

Dataset loaded successfully.
Dataset shape: (557646, 79)


In [15]:
df[' Label'].value_counts()

,count
Label,
DoS Hulk,231073
PortScan,158930
DDoS,128027
DoS GoldenEye,10293
FTP-Patator,7938
SSH-Patator,5897
DoS slowloris,5796
DoS Slowhttptest,5499
Bot,1966


In [16]:
print("--- Original Class Distribution ---")
print(df[' Label'].value_counts())
web_attack_map = {
    'Web Attack � Brute Force': 'Web_Attack',
    'Web Attack � XSS': 'Web_Attack',
    'Web Attack � Sql Injection': 'Web_Attack'
}

rare_attack_map = {
    'Infiltration': 'Rare_Attack',
    'Heartbleed': 'Rare_Attack'
}

df[' Label'] = df[' Label'].replace(web_attack_map)
df[' Label'] = df[' Label'].replace(rare_attack_map)

print("\n--- New Class Distribution After Grouping ---")
print(df[' Label'].value_counts())

--- Original Class Distribution ---
 Label
DoS Hulk                      231073
PortScan                      158930
DDoS                          128027
DoS GoldenEye                  10293
FTP-Patator                     7938
SSH-Patator                     5897
DoS slowloris                   5796
DoS Slowhttptest                5499
Bot                             1966
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Infiltration                      36
Web Attack � Sql Injection        21
Heartbleed                        11
Name: count, dtype: int64

--- New Class Distribution After Grouping ---
 Label
DoS Hulk            231073
PortScan            158930
DDoS                128027
DoS GoldenEye        10293
FTP-Patator           7938
SSH-Patator           5897
DoS slowloris         5796
DoS Slowhttptest      5499
Web_Attack            2180
Bot                   1966
Rare_Attack             47
Name: count, dtype: int64


In [17]:
print("--- Distribution Before Final Grouping ---")
print(df[' Label'].value_counts())

dos_map = {
    'DoS Hulk': 'DoS_Attack',
    'DDoS': 'DoS_Attack',
    'DoS GoldenEye': 'DoS_Attack',
    'DoS slowloris': 'DoS_Attack',
    'DoS Slowhttptest': 'DoS_Attack'
}

brute_force_map = {
    'FTP-Patator': 'Brute_Force',
    'SSH-Patator': 'Brute_Force'
}

other_map = {
    'Bot': 'Other_Attack',
    'Rare_Attack': 'Other_Attack'
}
df[' Label'] = df[' Label'].replace(dos_map)
df[' Label'] = df[' Label'].replace(brute_force_map)
df[' Label'] = df[' Label'].replace(other_map)

print("\n--- Final, Improved Class Distribution ---")
print(df[' Label'].value_counts())

--- Distribution Before Final Grouping ---
 Label
DoS Hulk            231073
PortScan            158930
DDoS                128027
DoS GoldenEye        10293
FTP-Patator           7938
SSH-Patator           5897
DoS slowloris         5796
DoS Slowhttptest      5499
Web_Attack            2180
Bot                   1966
Rare_Attack             47
Name: count, dtype: int64

--- Final, Improved Class Distribution ---
 Label
DoS_Attack      380688
PortScan        158930
Brute_Force      13835
Web_Attack        2180
Other_Attack      2013
Name: count, dtype: int64


In [18]:
for col in df.select_dtypes(include=np.number).columns:
    df[col].fillna(df[col].median(), inplace=True)
print("Missing values handled.")

X = df.drop(' Label', axis=1)
y = df[' Label']

y, class_names = pd.factorize(y)
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")

/tmp/ipython-input-2175202250.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


Missing values handled.
Found 5 unique classes.


In [19]:
y, class_names = pd.factorize(y)
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")

y_one_hot = tf.keras.utils.to_categorical(y, num_classes=num_classes)
print("Labels have been one-hot encoded.")

Found 5 unique classes.
Labels have been one-hot encoded.


In [20]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)

for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)
print("Infinite and missing values in features handled.")

scaler2 = MinMaxScaler()
X_scaled = scaler2.fit_transform(X)
print("Features have been normalized using Min-Max scaling.")
joblib.dump(scaler2, 'scaler2.gz')

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_dict = dict(enumerate(class_weights))
print(f"Calculated class weights: {class_weights_dict}")
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_one_hot,
    test_size=0.2,
    random_state=42,
    stratify=y
)
print(f"Data split into {len(X_train)} training and {len(X_val)} validation samples.")

/tmp/ipython-input-97739884.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)


Infinite and missing values in features handled.
Features have been normalized using Min-Max scaling.
Calculated class weights: {0: np.float64(0.2929674694237801), 1: np.float64(0.7017504561756748), 2: np.float64(55.40447093889717), 3: np.float64(51.16018348623853), 4: np.float64(8.06138055655945)}
Data split into 446116 training and 111530 validation samples.


In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

model.summary()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath='best_multiclass_model.h5', monitor='val_loss', save_best_only=True)
print("\n--- Starting Model Training ---")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)
print("--- Model Training Finished ---")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        10,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,693 (73.02 KB)

 Trainable params: 18,693 (73.02 KB)

...

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - accuracy: 0.9534 - loss: 0.4487 - precision_1: 0.9761 - recall_1: 0.8943 - val_accuracy: 0.9964 - val_loss: 0.0300 - val_precision_1: 0.9967 - val_recall_1: 0.9929
Epoch 2/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9902 - loss: 0.0986 - precision_1: 0.9916 - recall_1: 0.9890

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 56s 4ms/step - accuracy: 0.9902 - loss: 0.0986 - precision_1: 0.9916 - recall_1: 0.9890 - val_accuracy: 0.9971 - val_loss: 0.0212 - val_precision_1: 0.9973 - val_recall_1: 0.9969
Epoch 3/20
13932/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9904 - loss: 0.1016 - precision_1: 0.9914 - recall_1: 0.9891

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - accuracy: 0.9904 - loss: 0.1016 - precision_1: 0.9914 - recall_1: 0.9891 - val_accuracy: 0.9963 - val_loss: 0.0207 - val_precision_1: 0.9965 - val_recall_1: 0.9960
Epoch 4/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.9918 - loss: 0.0854 - precision_1: 0.9927 - recall_1: 0.9908 - val_accuracy: 0.9960 - val_loss: 0.0314 - val_precision_1: 0.9964 - val_recall_1: 0.9956
Epoch 5/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 45s 3ms/step - accuracy: 0.9925 - loss: 0.0896 - precision_1: 0.9933 - recall_1: 0.9915 - val_accuracy: 0.9949 - val_loss: 0.0394 - val_precision_1: 0.9951 - val_recall_1: 0.9946
Epoch 6/20
13938/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9921 - loss: 0.0838 - precision_1: 0.9930 - recall_1: 0.9904

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 0.9921 - loss: 0.0838 - precision_1: 0.9930 - recall_1: 0.9904 - val_accuracy: 0.9963 - val_loss: 0.0198 - val_precision_1: 0.9964 - val_recall_1: 0.9962
Epoch 7/20
13930/13942 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9917 - loss: 0.0858 - precision_1: 0.9928 - recall_1: 0.9903

13942/13942 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 0.9917 - loss: 0.0858 - precision_1: 0.9928 - recall_1: 0.9903 - val_accuracy: 0.9971 - val_loss: 0.0130 - val_precision_1: 0.9979 - val_recall_1: 0.9970
Epoch 8/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 0.9934 - loss: 0.0815 - precision_1: 0.9944 - recall_1: 0.9918 - val_accuracy: 0.9964 - val_loss: 0.0148 - val_precision_1: 0.9964 - val_recall_1: 0.9962
Epoch 9/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.9920 - loss: 0.0984 - precision_1: 0.9933 - recall_1: 0.9902 - val_accuracy: 0.9968 - val_loss: 0.0222 - val_precision_1: 0.9969 - val_recall_1: 0.9965
Epoch 10/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 43s 3ms/step - accuracy: 0.9925 - loss: 0.0746 - precision_1: 0.9938 - recall_1: 0.9904 - val_accuracy: 0.9965 - val_loss: 0.0144 - val_precision_1: 0.9967 - val_recall_1: 0.9965
Epoch 11/20
13942/13942 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 0.9931 - loss: 0.1068 - precision_1: 0.9940 - re

In [24]:
y_pred_proba = model.predict(X_val)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_val, axis=1)

print("\n--- Model Evaluation on Validation Set ---")
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

3486/3486 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step

--- Model Evaluation on Validation Set ---
Confusion Matrix:
[[75932     0    24    34   148]
 [   47 31722     0     4    13]
 [    1     0   401     0     1]
 [    7     0     0   390    39]
 [    9     0     0     2  2756]]


### SAMPLE TESTING

In [25]:
df=pd.read_csv('secondary_classification_dataset.csv')
X=df.drop(" Label",axis=1)
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)
print("Infinite and missing values in features handled.")
X_scaled_df = pd.DataFrame(X, columns=X.columns)

Infinite and missing values in features handled.


/tmp/ipython-input-3590754152.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)


In [26]:
first_row_list = X_scaled_df.iloc[0].values.tolist()

print("\nValues of the first row as a list:")
print(first_row_list)


Values of the first row as a list:
[80.0, 1293792.0, 3.0, 7.0, 26.0, 11607.0, 20.0, 0.0, 8.666666667, 10.26320288, 5840.0, 0.0, 1658.142857, 2137.29708, 8991.398927, 7.72921768, 143754.6667, 430865.8067, 1292730.0, 2.0, 747.0, 373.5, 523.9661249, 744.0, 3.0, 1293746.0, 215624.3333, 527671.9348, 1292730.0, 2.0, 0.0, 0.0, 0.0, 0.0, 72.0, 152.0, 2.318765304, 5.410452376, 0.0, 5840.0, 1057.545455, 1853.437529, 3435230.673, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1163.3, 8.666666667, 1658.142857, 72.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 26.0, 7.0, 11607.0, 8192.0, 229.0, 2.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [28]:
model1 = load_model('best_model.h5', compile=False)
model2 = load_model('best_multiclass_model.h5', compile=False)
print("Models loaded successfully.")

try:
    scaler1 = joblib.load('scaler1.gz')
    scaler2 = joblib.load('scaler2.gz')
    print("Scalers loaded successfully.")
except FileNotFoundError:
    print("Error: Scaler files not found. Please ensure 'scaler1.gz' and 'scaler2.gz' exist.")
    exit()

x_raw = np.array([first_row_list])
x_input_scaled1 = scaler1.transform(x_raw)
output1 = model1.predict(x_input_scaled1)
print(f"\nModel 1 (Binary) raw output: {output1[0][0]:.4f}")

if output1[0][0] > 0.5:
    print("Attack detected. Proceeding to Stage 2 for multiclass classification...")
    x_input_scaled2 = scaler2.transform(x_raw)
    output2 = model2.predict(x_input_scaled2)
    predicted_class_index = np.argmax(output2, axis=1)[0]

    print(f"\nModel 2 (Multiclass) raw output (probabilities): {output2[0]}")
    print(f"Final Predicted Class Index: {predicted_class_index}")

else:
    print("\nResult: Benign data. No further action needed.")

Models loaded successfully.
Scalers loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step

Model 1 (Binary) raw output: 1.0000
Attack detected. Proceeding to Stage 2 for multiclass classification...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step

Model 2 (Multiclass) raw output (probabilities): [9.9997735e-01 2.2688186e-05 2.2147821e-09 2.6475639e-09 4.8972613e-16]
Final Predicted Class Index: 0
